학습의 진행상황을 한 눈에 볼 수 있게 도와주는 Tool:  
## TensorBoard  
기존의 번잡한 print함수의 반복을 쓰지 않고 깔끔하게 출력을 볼 수 있다.  


Step1: From TF graph, decide which tensors you want to log  
어떤 함수를 로깅할것인지 정한다. tf.summary.histyogram(), tf.summary.scalar()

Step2: Merge all summaries  
병합하여 summary정보를 만든다. tf.summary.merge_all()

Step3: Create writer and add graph  
파일로써 기록할 장소를 정한다.  
tf.summary.FileWriter()  
그래프를 추가시킨다.  
add_graph() 

Step4: Run summary merge and add_summary  
summary를 추가하고 실행한다. feed_dict 이용  
add_summary()  
step 값도 입력해줄 수 있다.  

Step5: Lauch TensorBoard  
파일이 저장된 위치에서 터미널을 이용해 텐서보드를 실행

## Step 1:
- 보려는 것(ex. cost)이 스칼라(0차 텐서)이면 tf.summary.scalar()를,  
- 여러 개(ex. Weight, bias, hypothesis)이면 tf.summary.histogram()을 이용한다.
### Graph hierarchy 보기: TensorFlow Name Scope
tf.name_scope()를 이용해서 그래프를 볼 때 레이어를 정의해줄 수 있다. > 보기 편해진다.  

In [1]:
# Lab 9 XOR
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name="x")
Y = tf.placeholder(tf.float32, [None, 1], name="y")

#Adding Scopes
with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.random_normal([2, 2]), name="weight_1")
    b1 = tf.Variable(tf.random_normal([2]), name="bias_1")
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    tf.summary.histogram("W1", W1)
    tf.summary.histogram("b1", b1)
    tf.summary.histogram("Layer1", layer1)


with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name="weight_2")
    b2 = tf.Variable(tf.random_normal([1]), name="bias_2")
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    tf.summary.histogram("W2", W2)
    tf.summary.histogram("b2", b2)
    tf.summary.histogram("Hypothesis", hypothesis)

# cost/loss function
with tf.name_scope("Cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    tf.summary.scalar("Cost", cost)

with tf.name_scope("Train"):
    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)



<tf.Tensor 'accuracy:0' shape=() dtype=string>

### Step 2~3: Merge summaries and create writer after creating session
### Step 4: Run merged summary and write(add_summary)

In [3]:
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        #Step 4: Run merged summary and write(add_summary)
        _, summary, cost_val = sess.run(
            [train, merged_summary, cost], feed_dict={X: x_data, Y: y_data}
        )
        writer.add_summary(summary, global_step=step) #step value: 경과를 관찰할 지점의 개수

        if step % 100 == 0:
            print(step, cost_val)

    # Accuracy report
    h, p, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data}
    )
    
    print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")

0 0.7168676
100 0.69085675
200 0.4987003
300 0.22536185
400 0.122916736
500 0.0791458
600 0.05615393
700 0.042352103
800 0.033301707
900 0.026986767
1000 0.022373727
1100 0.018883027
1200 0.016166903
1300 0.014005062
1400 0.01225182
1500 0.010807454
1600 0.009601359
1700 0.008582579
1800 0.0077132024
1900 0.006964777
2000 0.0063154213
2100 0.0057480196
2200 0.0052491585
2300 0.0048080757
2400 0.0044160266
2500 0.0040659807
2600 0.003752165
2700 0.0034696562
2800 0.003214465
2900 0.002983223
3000 0.0027729843
3100 0.0025813607
3200 0.0024062647
3300 0.0022458504
3400 0.0020985582
3500 0.0019630534
3600 0.0018381808
3700 0.0017228625
3800 0.0016161547
3900 0.0015173084
4000 0.0014256047
4100 0.0013404004
4200 0.0012611565
4300 0.0011873196
4400 0.0011184708
4500 0.0010541915
4600 0.0009941373
4700 0.0009379648
4800 0.00088534516
4900 0.00083605415
5000 0.00078983756
5100 0.0007464565
5200 0.00070570176
5300 0.00066743896
5400 0.000631429
5500 0.00059755263
5600 0.00056566036
5700 0.00053

Step 5: Lauch Tensorboard(local)

`writer=tf.summary.FileWriter("./logs/xor_logs")`에서 정의한 경로로 로그파일이 만들어지므로
`cmd창에 tensorboard --logdir=./logs/xor_logs 를 실행해줌
(이 파일에서는 폴더명이 xor_logs_r0_01이므로 유의)


---
- 에러 발생함. 경로 문제인지?
tensorboard could not load dynamic library 'cudart4_100.dll' 에러

이 PC에서는,
C:\GitHub\LearningProgrammingLanguages\TensorFlow\Lectures\SungKim\logs\xor_logs_r0_01
이므로
C:/GitHub/LearningProgrammingLanguages/TensorFlow/Lectures/SungKim/logs/xor_logs_r0_01


- 해결책?  
https://www.joe0.com/2019/10/19/how-resolve-tensorflow-2-0-error-could-not-load-dynamic-library-cudart64_100-dll-dlerror-cudart64_100-dll-not-found/
